In [31]:
# 1. Install required packages
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1
!pip install pandas openpyxl | tail -n 1


In [32]:
# 2. Imports and credentials
import os, types, pandas as pd
from botocore.client import Config
import ibm_boto3
from sklearn.model_selection import train_test_split
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


In [33]:
# 3. IBM Cloud credentials
url = "https://us-south.ml.cloud.ibm.com"
apikey = "8SzgMoHFT5MfqbhGJe4jKLzSBNknFzmyLgjg9rKkkNt8"
credentials = {"url": url, "apikey": apikey}

In [34]:
# 4. Get project ID
try: project_id = os.environ["PROJECT_ID"]
except KeyError: project_id = input("Enter your project_id: ")


In [35]:
# 5. Load data from IBM COS
cos = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id=apikey,
    ibm_auth_endpoint='https://iam.cloud.ibm.com/oidc/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.us-south.cloud-object-storage.appdomain.cloud'
)

bucket = 'hotel-review-sentimentanaysisbucket-ocwh0le9s0qj4ax'
object_key = 'hotel_reviews.csv'  # Make sure this file exists with "Review" and "Sentiment" columns

body = cos.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "__iter__"):
    from types import MethodType
    def _iter_(self): yield from self.read().splitlines()
    body._iter_ = MethodType(_iter_, body)

data = pd.read_csv(body)
data.columns = data.columns.str.strip()

# Optional label mapping for display
label_map = {-1: 'negative', 0: 'neutral', 1: 'positive'}
if 'Sentiment' in data.columns:
    data['SentimentLabel'] = data['Sentiment'].map(label_map)


In [36]:
# 6. Split data
train, test = train_test_split(
    data, test_size=0.3, random_state=33, stratify=data['sentiment']
)


In [37]:
# 7. Load watsonx.ai model
model = Model(
    model_id=ModelTypes.FLAN_T5_XXL,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.RANDOM_SEED: 33,
        GenParams.REPETITION_PENALTY: 1,
        GenParams.MIN_NEW_TOKENS: 3,
        GenParams.MAX_NEW_TOKENS: 20
    },
    credentials=credentials,
    project_id=project_id
)


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [38]:
# 8. Prompt Template
instruction = """
Classify the hotel review into:
1. A service category (e.g., food, cleanliness, room service, staff, location, etc.)
2. The sentiment (positive, negative, or neutral)

Format:
category: <category>
sentiment: <sentiment>
"""

In [44]:
# 9. Function to analyze user input
def classify_review(review_text):
    prompt = f"{instruction}\nreview: {review_text}\noutput:"
    response = model.generate(prompt)
    result = response["results"][0]["generated_text"]
    return result


In [45]:
# 10. Batch classification using TRAINING samples
from tqdm import tqdm

def batch_classify_reviews(df, review_column="Review"):
    reviews = df[review_column].tolist()
    predictions = []

    for review in tqdm(reviews, desc="Processing reviews"):
        prompt = f"{instruction}\nreview: {review}\noutput:"
        response = model.generate(prompt)
        predictions.append(response["results"][0]["generated_text"].strip())

    return predictions

In [46]:
# 11. Select a subset of training data for batch testing (e.g., 10 samples)
sample_train = train.sample(10, random_state=42).reset_index(drop=True)



In [48]:
# 12. Run predictions
sample_train['Prediction'] = batch_classify_reviews(sample_train, review_column='review')



Processing reviews: 100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


In [56]:
print("Raw Predictions from Model:")
print(sample_train[['review', 'Prediction']])


Raw Predictions from Model:
                                 review                       Prediction
0  Nothing exceptional about the staff.                  negative. staff
1              Location was manageable.                        neutral..
2        Polite and professional staff.                  positive. staff
3             Satisfactory cleanliness.            cleanliness positive.
4             Expected service quality.                       positive..
5                 Average food overall.                  food neutrality
6            Decent spot for the price.                       positive..
7            Decent spot for the price.                       positive..
8    Room was too small and poorly lit.          negative category: room
9    Clean towels and sheets every day.  positive. category: cleanliness


In [59]:
user_review = input("Enter your hotel review: ")
result = classify_review(user_review)

print("\n Classified Result: ")
print(result)

Enter your hotel review:  The room was too small and dimly lit.



 Classified Result: 
negative category: room


In [60]:
user_review = input("Enter your hotel review: ")
result = classify_review(user_review)

print("\n Classified Result: ")
print(result)

Enter your hotel review:  Excellent breakfast and friendly staff.



 Classified Result: 
positive. food


In [61]:
user_review = input("Enter your hotel review: ")
result = classify_review(user_review)

print("\n Classified Result: ")
print(result)

Enter your hotel review:  Average cleanliness in the bathroom.



 Classified Result: 
cleanliness neutrality


In [62]:
user_review = input("Enter your hotel review: ")
result = classify_review(user_review)

print("\n Classified Result: ")
print(result)

Enter your hotel review:  the food was great but the staff behavior was not good



 Classified Result: 
negative staff attitude
